In [15]:
from collections import Counter
import numpy as np
np.random.seed(42)
import pandas as pd

import cv2
from sklearn.model_selection import KFold
from sklearn.metrics import fbeta_score

from keras.models import Model
#from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg19 import VGG19
from keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D,Flatten
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [18]:
#input_size = 128
#input_channels = 3
epochs = 1
batch_size = 32
n_folds = 2
training = True
ensemble_voting = False  # If True, use voting for model ensemble, otherwise use averaging

In [3]:
# Load data
train = pd.read_json("data/iceberg/train.json")
test = pd.read_json("data/iceberg/test.json")

In [4]:
# Train data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train['band_1']])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train['band_2']])

X_train = np.concatenate([x_band1[:, :, :, np.newaxis],
                          x_band2[:, :, :, np.newaxis],
                          ((x_band1+x_band1)/2)[:, :, :, np.newaxis]], axis=-1)

target_train=train['is_iceberg']

del train

In [5]:
# Test data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test['band_1']])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test['band_2']])

X_test = np.concatenate([x_band1[:, :, :, np.newaxis],
                         x_band2[:, :, :, np.newaxis],
                         ((x_band1+x_band1)/2)[:, :, :, np.newaxis]], axis=-1)

id_test = test['id'].values

del test; del x_band1; del x_band2

In [23]:
# Define CNN Model Architecture
img_height = 224
img_width = 224
img_channels = 3
img_dim = (img_height, img_width, img_channels)

def Vgg19(img_dim=img_dim):
    input_tensor = Input(shape=img_dim)
    base_model = VGG19(include_top=False,
                       weights='imagenet',
                       input_shape=(img_height, img_width, img_channels))
    bn = BatchNormalization()(input_tensor)
    x = base_model(bn)
    #x = GlobalAveragePooling2D()(x)
    #x = Dropout(0.5)(x)
    x = Flatten()(x)
    output = Dense(1, activation='sigmoid')(x)
    model = Model(input_tensor, output)
    
    return model

model = Vgg19()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 224, 224, 3)       12        
_________________________________________________________________
vgg19 (Model)                (None, 7, 7, 512)         20024384  
_________________________________________________________________
flatten_3 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 25089     
Total params: 20,049,485
Trainable params: 20,049,479
Non-trainable params: 6
_________________________________________________________________


In [7]:
kf = KFold(n_splits=n_folds, shuffle=True, random_state=1)
fold_count = 0
#y_full_test = []
#thres_sum = np.zeros(2, np.float32)
img_size = (img_height, img_width)


In [ ]:
for train_index, test_index in kf.split(X_train):

    fold_count += 1
    print('Fold ', fold_count)   


    def augment(src, choice):
            if choice == 0:
                # Rotate 90
                src = np.rot90(src, 1)
            if choice == 1:
                # flip vertically
                src = np.flipud(src)
            if choice == 2:
                # Rotate 180
                src = np.rot90(src, 2)
            if choice == 3:
                # flip horizontally
                src = np.fliplr(src)
            if choice == 4:
                # Rotate 90 counter-clockwise
                src = np.rot90(src, 3)
            if choice == 5:
                # Rotate 180 and flip horizontally
                src = np.rot90(src, 2)
                src = np.fliplr(src)
            return src
    
    df_train = X_train[train_index]
    y_train = target_train[train_index]
    #print(df_train[:3])
    if training:
        print('Training on {} samples'.format(len(df_train)))
        print('Training target on {} samples'.format(len(y_train)))



    def train_generator():
        while True:
            for start in range(0, len(df_train), batch_size):
                x_batch = []
                end = min(start + batch_size, len(df_train))
                y_batch = y_train[start:end]
                for img in df_train[start:end]:
                    #img = cv2.imread('input/train-jpg/{}.jpg'.format(f))
                    new_img = cv2.resize(img, img_size)
                    new_img = augment(new_img, np.random.randint(6))
                    x_batch.append(new_img)                    
                x_batch = np.array(x_batch, np.float32)/ 255.
                y_batch = np.array(y_batch, np.uint8)
                yield x_batch, y_batch


    df_valid = X_train[test_index]
    y_valid = target_train[test_index]
    #print(df_valid[:3])
    print('Validating on {} samples'.format(len(df_valid)))


    def valid_generator():
        while True:
            for start in range(0, len(df_valid), batch_size):
                x_batch = []
                end = min(start + batch_size, len(df_valid))
                y_batch = y_valid[start:end]
                for img in df_valid[start:end]:
                    new_img = cv2.resize(img, img_size)
                    x_batch.append(new_img)
                x_batch = np.array(x_batch, np.float32)
                y_batch = np.array(y_batch, np.uint8)
                yield x_batch, y_batch

    def test_generator(transformation):
        while True:
            for start in range(0, len(X_test), batch_size):
                x_batch = []
                end = min(start + batch_size, len(X_test))
                for img in X_test[start:end]:
                    new_img = cv2.resize(img, img_size)
                    new_img = augment(img, transformation)
                    x_batch.append(new_img)
                x_batch = np.array(x_batch, np.float32)
                yield x_batch

    callbacks = [EarlyStopping(monitor='val_loss',
                               patience=4,
                               verbose=1,
                               min_delta=1e-4),
                 ReduceLROnPlateau(monitor='val_loss',
                                   factor=0.1,
                                   patience=2,
                                   cooldown=2,
                                   verbose=1),
                 ModelCheckpoint(filepath='best_weights.fold_' + str(fold_count) + '.hdf5',
                                 save_best_only=True,
                                 save_weights_only=True)]

    model = model

    model.compile(loss='binary_crossentropy',optimizer=Adam(lr=1e-4),metrics=['accuracy'])
    
    train_steps = len(df_train) / batch_size
    valid_steps = len(df_valid) / batch_size
    test_steps = len(X_test) / n_folds

    if training:
        model.fit_generator(generator=train_generator(),
                            steps_per_epoch=train_steps,
                            epochs=epochs,
                            verbose=2,
                            callbacks=callbacks,
                            validation_data=valid_generator(),
                            validation_steps= valid_steps)


    # Load best weights
    model.load_weights(filepath='best_weights.fold_' + str(fold_count) + '.hdf5')

    p_valid = model.predict_generator(generator=valid_generator(),steps= valid_steps)
    
#     print('----------------------------------------')
#     print('Running train evaluation on fold {}'.format(i))
#     train_score = model.evaluate_generator(train_generator(), steps=train_steps)        
#     print('Running validation evaluation on fold {}'.format(i))
#     valid_score = model.evaluate_generator(valid_generator(), steps=valid_steps)
#     print('----------------------------------------')   
        
#     print('Train loss: {:0.5f}\n Train acc: {:0.5f} for fold {}'.format(train_score[0],
#                                                                             train_score[1], i))
#     print('Valid loss: {:0.5f}\n Valid acc: {:0.5f} for fold {}'.format(valid_score[0],
#                                                                             valid_score[1], i))
#     print('----------------------------------------')

#     train_scores.append(train_score[1])
#     valid_scores.append(valid_score[1])
#     print('Avg Train Acc: {:0.5f}\nAvg Valid Acc: {:0.5f} after {} folds'.format
#               (np.mean(train_scores), np.mean(valid_scores), i))
#     print('----------------------------------------')
    
#     print('Running test predictions with fold {}'.format(i))        
#     preds_test_fold = model.predict_generator(generator=test_generator(),
#                                               steps=test_steps, verbose=1)[:, -1]
#     print(preds_test_fold)

#     preds_test += preds_test_fold

#     print('\n\n')

#     i += 1

#     if i <= n_fold:
#         print('Now beginning training for fold {}\n\n'.format(i))
#     else:
#         print('Finished training!')
    

    
#     # 6-fold TTA
#     number_of_times = 2
#     p_full_test = []
#     for i in range(number_of_times):
#         p_test = model.predict_generator(generator=test_generator(transformation=i),steps= test_steps)
#         p_full_test.append(p_test)
        
#     preds = p_full_test/ number_of_times

#     p_test = np.array(p_full_test[0])
#     for i in range(1, 6):
#         p_test += np.array(p_full_test[i])
#     p_test /= 6

#     y_full_test.append(p_test)

# result = np.array(y_full_test[0])
# if ensemble_voting:
#     for f in range(len(y_full_test[0])):  # For each file
#         for tag in range(17):  # For each tag
#             preds = []
#             for fold in range(n_folds):  # For each fold
#                 preds.append(y_full_test[fold][f][tag])
#             pred = Counter(preds).most_common(1)[0][0]  # Most common tag prediction among folds
#             result[f][tag] = pred
# else:
#     for fold in range(1, n_folds):
#         result += np.array(y_full_test[fold])
#     result /= n_folds
# result = pd.DataFrame(result, columns=labels)

# preds = []
# thres = (thres_sum / n_folds).tolist()

# for i in result.shape[0]:
#     a = result.ix[[i]]
#     a = a.apply(lambda x: x > thres, axis=1)
#     a = a.transpose()
#     a = a.loc[a[i] == True]
#     ' '.join(list(a.index))
#     preds.append(' '.join(list(a.index)))

Fold  12
Training on 1283 samples
Training target on 1283 samples
Validating on 321 samples
Epoch 1/1
101s - loss: 0.4663 - acc: 0.7915 - val_loss: 4.3846 - val_acc: 0.6168
Fold  13
Training on 1283 samples
Training target on 1283 samples
Validating on 321 samples
Epoch 1/1
102s - loss: 0.4087 - acc: 0.7976 - val_loss: 1.9505 - val_acc: 0.7664
Fold  14
Training on 1283 samples
Training target on 1283 samples
Validating on 321 samples
Epoch 1/1


In [ ]:
X_test['is_iceberg'] = p_valid
#X_test.to_csv('submission.csv', index=False)
X_test.head(5)